# ASP Bundle Adjust Plotting
## Examples for BlackSky Easton Glacier test case (n=20)
David Shean  
12/24/22

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as cx

In [ ]:
import matplotlib.colors

In [ ]:
topdir = '/Users/dshean/scr/BlackSky/EastonGlacier_20220918-20221012/non-ortho'

In [ ]:
cd $topdir

In [ ]:
#ba_prefix = 'ba_all/ba_all'

In [ ]:
ba_prefix = 'ba_all/ba_all_tri_weight'

## Plot convergence angles

In [ ]:
conv_txt = ba_prefix+'-convergence_angles.txt'

In [ ]:
conv_cols = ['img1','img2','conv_25','conv_50','conv_75','num_angles']
conv = pd.read_csv(conv_txt, delimiter=' ', skiprows=1, header=0, names=conv_cols, index_col=False)

In [ ]:
conv_valid = conv[conv['num_angles'] != 0]

In [ ]:
conv_valid.reset_index().plot.scatter(x='index', y='conv_50', c='num_angles', cmap='inferno')

In [ ]:
f, ax = plt.subplots()
m = ax.scatter(conv_valid.index, conv_valid['conv_50'], c=conv_valid['num_angles'], norm=matplotlib.colors.LogNorm())
plt.colorbar(m)

## Plot camera positions

In [ ]:
cam_init_csv = ba_prefix+'-initial-cameras.csv'
cam_final_csv = ba_prefix+'-final-cameras.csv'

In [ ]:
cam_cols=['input_cam_file', 'x','y','z','r11','r12','r13','r21','r22','r23','r31','r32','r33']
cam_init = pd.read_csv(cam_init_csv, header=0, names=cam_cols, index_col='input_cam_file')
cam_final = pd.read_csv(cam_final_csv, header=0, names=cam_cols, index_col='input_cam_file')

In [ ]:
cam_delta = cam_init - cam_final

In [ ]:
cam_delta['m'] = np.sqrt(np.square(cam_delta[['x','y','z']]).sum(axis=1))

In [ ]:
cam_final.index.to_series().str.split('-', expand=True)[1]

In [ ]:
global_id = cam_final.index.to_series().str.split('-', expand=True)[1].astype('int') - 100

In [ ]:
#cam_init_idx = cam_init['input_cam_file'].str.split('/', expand=True)

In [ ]:
cam_init_gdf = gpd.GeoDataFrame(cam_init, geometry=gpd.points_from_xy(cam_init['x'], cam_init['y'], cam_init['z'], crs='EPSG:4978'))
cam_final_gdf = gpd.GeoDataFrame(cam_final, geometry=gpd.points_from_xy(cam_init['x'], cam_init['y'], cam_init['z'], crs='EPSG:4978'))

In [ ]:
cam_final_gdf['m'] = cam_delta['m']
cam_final_gdf['global_id'] = global_id

In [ ]:
cam_final_gdf

In [ ]:
map_crs = 'EPSG:32610'

In [ ]:
ax = cam_init_gdf.to_crs(map_crs).plot(color='r')
cam_final_gdf.to_crs(map_crs).plot(ax=ax, color='b')
cx.add_basemap(ax, crs=map_crs)
#ax.set_aspect('equal')

In [ ]:
ax = cam_final_gdf.to_crs(map_crs).plot(column='global_id', cmap='tab20', legend='True', legend_kwds={'label': "BlackSky Satellite ID"})
cx.add_basemap(ax, crs=map_crs)

In [ ]:
ax = cam_final_gdf.to_crs(map_crs).plot(column='m', norm=matplotlib.colors.LogNorm(), legend='True', legend_kwds={'label': "Translation Magnitude (m)"})
cx.add_basemap(ax, crs=map_crs)

In [ ]:
cam_delta['m'].plot.bar()

### Compute rotation delta magnitude

In [ ]:
from scipy.spatial.transform import Rotation as R

## Residuals

In [ ]:
def read_residuals(csv_fn):
    resid_cols=['lon', 'lat', 'height_above_datum', 'mean_residual', 'num_observations']
    resid_df = pd.read_csv(csv_fn, skiprows=2, names=resid_cols)
    resid_df['from_DEM'] = resid_df['num_observations'].str.contains('# from DEM')
    resid_df['num_observations'] = resid_df['num_observations'].str.split('#', expand=True)[0].astype(int)
    resid_gdf = gpd.GeoDataFrame(resid_df, geometry=gpd.points_from_xy(resid_df['lon'], resid_df['lat'], crs='EPSG:4326'))
    return resid_gdf

In [ ]:
resid_init_csv = ba_prefix+'-initial_residuals_pointmap.csv'
resid_init = read_residuals(resid_init_csv)

In [ ]:
resid_final_csv = ba_prefix+'-final_residuals_pointmap.csv'
resid_final = read_residuals(resid_final_csv)

In [ ]:
resid_init.describe()

In [ ]:
resid_final.describe()

In [ ]:
def resid_plot(col='mean_residual', clip_final=True, lognorm=False):
    f, axa = plt.subplots(1,2, sharex=True, sharey=True)
    vmin = min(resid_init[col].min(), resid_final[col].min())
    vmax = max(resid_init[col].max(), resid_final[col].max())
    norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)
    if lognorm:
        norm = matplotlib.colors.LogNorm(vmin=vmin, vmax=vmax)
    plot_kw = {'norm':norm, 's':1, 'legend':True, 'legend_kwds':{'label': col}}
    resid_final.sort_values(by=col).to_crs(map_crs).plot(ax=axa[1], column=col, **plot_kw)
    cx.add_basemap(ax=axa[1], crs=map_crs, attribution_size=0)
    if clip_final:
        axa[0].autoscale(False)
    resid_init.sort_values(by=col).to_crs(map_crs).plot(ax=axa[0], column=col, **plot_kw)
    cx.add_basemap(ax=axa[0], crs=map_crs, attribution_size=0)
    #plt.tight_layout()

In [ ]:
resid_plot(col='mean_residual', lognorm=True)

In [ ]:
resid_plot(col='num_observations')

## Mapproject Residuals

In [ ]:
def read_mapproj_match_offset(csv_fn):
    resid_cols=['lon', 'lat', 'height_above_datum', 'mapproj_ip_dist_meters']
    resid_df = pd.read_csv(csv_fn, skiprows=2, names=resid_cols)
    resid_gdf = gpd.GeoDataFrame(resid_df, geometry=gpd.points_from_xy(resid_df['lon'], resid_df['lat'], crs='EPSG:4326'))
    return resid_gdf

In [ ]:
mapproj_match_offset_txt = ba_prefix+'-mapproj_match_offsets.txt'
mapproj_match_offset = read_mapproj_match_offset(mapproj_match_offset_txt)

In [ ]:
mapproj_match_offset.describe()

In [ ]:
col='mapproj_ip_dist_meters'

In [ ]:
mapproj_match_offset.sort_values(by=col).to_crs(map_crs).plot(column=col, norm=matplotlib.colors.LogNorm(), legend=True)

## Geoplot tests for KDE

In [ ]:
import geoplot as gplt
import geoplot.crs as gcrs

In [ ]:
ax = gplt.pointplot(mapproj_match_offset, projection=gcrs.AlbersEqualArea(), s=1)
gplt.kdeplot(mapproj_match_offset[['mapproj_ip_dist_meters','geometry']], projection=gcrs.AlbersEqualArea(), ax=ax)